# Generate financial time series (FTS) data

In [1]:
import re
import sys
import warnings
import pandas as pd # type: ignore
import yfinance as yf # type: ignore

sys.path.append('../modules')
import get_financial_time_series as get_fts # type: ignore

warnings.filterwarnings("ignore")
pd.options.mode.chained_assignment = None
pd.set_option('display.max_columns', None)

## Global variables

In [2]:
input_path_raw = "../input_files/raw_data"
input_path_processed = "../input_files/processed_data"
input_path_data_dictionary = "../input_files/data_dictionary"
output_path = "../output_files"
input_generation_date = "2024-06-24"

In [4]:
# Standard and Poor's 500 - S&P 500 (United States)
# Source https://en.wikipedia.org/wiki/List_of_S%26P_500_companies
# "2000-01-01" - "2022-12-31"
# GEV: Data doesn't exist for startDate = 946702800, endDate = 1672462800")
# KVUE: Data doesn't exist for startDate = 946702800, endDate = 1672462800")
# SOLV: Data doesn't exist for startDate = 946702800, endDate = 1672462800")
# VLTO: Data doesn't exist for startDate = 946702800, endDate = 1672462800")

#df_stock_index = pd.read_csv("{}/sp500_tickers.csv".format(input_path_data_dictionary), low_memory = False)

# Mexico Price and Quote Index (Mexico)
# source https://es.finance.yahoo.com/quote/%5EMXX/components/?guccounter=1&guce_referrer=aHR0cHM6Ly93d3cuZ29vZ2xlLmNvbS8&guce_referrer_sig=AQAAAICc-QDC-sGwCChRyZU_MzY5IZIpLgF6EOVbbq-P7L_F6E0ur0Zz-MAwE1zXvNewk1qXCrCh4tU6A48XLFTPTaxgwdtcHisQnftQhKrJjlhLFjzRswWW8ybr08epnyjiWfra8DQjOdPTIIBrNB86THdyO7MnRmHpizHkFyNPL1Np
# "2005-01-03" - "2022-12-31"

#df_stock_index = pd.read_csv("{}/ipc_mexico_tickers.csv".format(input_path_data_dictionary), low_memory = False)

# DAX Performance Index (Germany)
# source https://markets.businessinsider.com/index/components/dax
# "2001-01-01" - "2022-12-31"

df_stock_index = pd.read_csv("{}/dax_tickers.csv".format(input_path_data_dictionary), low_memory = False)

list_stock_indexes = df_stock_index.set_index("ticker")["ticker_name"].to_dict()

## Download time series - Stock indexes

In [5]:
df_market = get_fts.process_financial_time_series(
    ticker_dict = list_stock_indexes,
    initial_date = "2000-01-01",
    final_date = "2024-12-31",#input_generation_date,
    interval = "1d"
)

----------------------------------- YAHOO FINANCE DATA -----------------------------------

- Download 1COV.DE with initial 2238 rows and 2191 rows after profiling
- Processed 1COV.DE : COVESTRO
- Download ADS.DE with initial 6280 rows and 6124 rows after profiling
- Processed ADS.DE : ADIDAS
- Download AIR.DE with initial 6280 rows and 6009 rows after profiling
- Processed AIR.DE : AIRBUS
- Download ALV.DE with initial 6280 rows and 6144 rows after profiling
- Processed ALV.DE : ALLIANZ
- Download BAS.DE with initial 6280 rows and 6161 rows after profiling
- Processed BAS.DE : BASF
- Download BAYN.DE with initial 6280 rows and 6094 rows after profiling
- Processed BAYN.DE : BAYER
- Download BEI.DE with initial 6280 rows and 6141 rows after profiling
- Processed BEI.DE : BEIERSDORF
- Download BMW.DE with initial 6280 rows and 6157 rows after profiling
- Processed BMW.DE : BMW
- Download BNR.DE with initial 3640 rows and 3573 rows after profiling
- Processed BNR.DE : BRENNTAG
- Download

## Save data in input files for no reprocessing

In [6]:
# Standard and Poor's 500 - S&P 500 (United States)
#df_market.to_pickle("{}/df_stock_index_gspc_{}.pkl".format(input_path_processed, re.sub("-", "", input_generation_date)))

# Mexico Price and Quote Index (Mexico)
#df_market.to_pickle("{}/df_stock_index_mxx_{}.pkl".format(input_path_processed, re.sub("-", "", input_generation_date)))

# DAX Performance Index (Germany)
df_market.to_pickle("{}/df_stock_index_gdaxi_{}.pkl".format(input_path_processed, re.sub("-", "", input_generation_date)))

df_market

,date,symbol,Adj Close,return,log_return,z_score_log_return
0,2015-10-07,1COV.DE,20.616699,-0.078093,0.000000,0.000000
1,2015-10-08,1COV.DE,20.288704,-0.327995,-0.016037,-0.778247
2,2015-10-09,1COV.DE,20.460512,0.171808,0.008432,0.384645
3,2015-10-12,1COV.DE,20.304325,-0.156187,-0.007663,-0.380274
4,2015-10-13,1COV.DE,20.070045,-0.234280,-0.011606,-0.567644
...,...,...,...,...,...,...
218320,2024-07-17,ZAL.DE,24.090000,-0.430000,-0.017692,-0.648976
218321,2024-07-18,ZAL.DE,24.250000,0.160000,0.006620,0.238710
218322,2024-07-19,ZAL.DE,23.290001,-0.959999,-0.040392,-1.477808
218323,2024-07-22,ZAL.DE,23.309999,0.019999,0.000858,0.028346
